In [23]:
import numpy as np

In [24]:
from torch import nn, Tensor
from dataclasses import dataclass


# hormone = 20 [id], 
# neuron = [activation_threshold, activation_decay, signal_multiplier, signal_delay, connection_affinity, spontaneity, ambient_hormone_release_rate, activation_hormone_release, hormone_decay_rate, hormone_range]
#          [offsets...] x 8 max simultaneous hormone effects
#          [decay_rates...] x 8
#          (d x 10) NL (10 x 20) - offsets
#          

# neuron_division_matrix = (d x d)

@dataclass
class Neuron:
  # FUNCTIONAL PARAMETERS
  # - activation parameters
  activation_threshold: float  # positive
  activation_cooldown: float  # positive
  signal_strength: float  # can be excitatory or inhibitory (positive or negative)
  
  # included in internal state as well (it should be taken into account but cannot be derived solely from current internal state)
  mitosis_qualification: float  # when this reaches 1.0, mitosis occurs. updates increment this, and it is set back to 0 upon mitosis
  
  # INTERNAL STATE
  latent_state: Tensor  # produced exclusively by transforms defined in the genome
  hormone_influence: Tensor  # influence by hormones emitted from other neurons
  total_receptivity: float  # summed connectivity coefficients of incoming connections
  total_emissivity: float  # summed connectivity coefficients of outgoing connections
  activation_progress: float  # incremented by signals
  
  # HIDDEN STATE (for implementation purposes only)
  position: Tensor  # 3d global position
  inputs: Tensor  # (MAX_CONNECTIONS x 2), each connection includes a neuron ID and a connectivity coefficient
  outputs: Tensor  # same structure as inputs
  
@dataclass
class Genome:
  pluripotent_latent_state: Tensor  # initial latent state for the first neuron(s)
  latent_to_parameters: nn.Module  # takes in internal state, outputs activation parameters
  passive_transform: nn.Module  # applied continuously
  activation_transform: nn.Module  # takes in internal state, outputs new latent state
  # hormone_transform: nn.Module  # takes in internal state and hormone, outputs new latent state
  hormone_emission: nn.Module  # takes in internal state, outputs hormone code and emission range
  hormone_decay: Tensor  # values in [0, 1] that are continuously multiplied against hormone_influence
  connectivity_coefficient: nn.Module  # takes in internal state pair and relative position (emitter and receiver and emitter to receiver), outputs receptivity of latter [0,1]
  mitosis_results: nn.Module  # takes in internal state, outputs new latent state and daughter latent state and mitosis direction
  
  # MUTABILITY
  # ... (modules of the same dimensions as genome which define the rate of genetic drift)
  

In [25]:
from typing import SupportsIndex, Union
# data formatting strategy for efficiency and compactness:
# 1. slot all neuron data into a single 1d tensor, and then stack each neuron tensor.
#    the position of a neuron tensor is its ID except for position 0, which is a dummy
#    for computational purposes
# 2. Genome can just live in a class
from enum import Enum, IntEnum


class _Properties(IntEnum):
    ACTIVATION_THRESHOLD     = 0
    ACTIVATION_COOLDOWN      = 1
    SIGNAL_STRENGTH          = 2
    LATENT_STATE             = 3
    HORMONE_INFLUENCE        = 4
    TOTAL_RECEPTIVITY        = 5
    TOTAL_EMISSIVITY         = 6
    ACTIVATION_PROGRESS      = 7
    MITOSIS_QUALIFICATION    = 8
    POSITION                 = 9
    INPUTS                   = 10
    OUTPUTS                  = 11


_DATA_SIZES = {
    _Properties.ACTIVATION_THRESHOLD:     1,
    _Properties.ACTIVATION_COOLDOWN:      1,
    _Properties.SIGNAL_STRENGTH:          1,
    _Properties.LATENT_STATE:             24,
    _Properties.HORMONE_INFLUENCE:        10,
    _Properties.TOTAL_RECEPTIVITY:        1,
    _Properties.TOTAL_EMISSIVITY:         1,
    _Properties.ACTIVATION_PROGRESS:      1,
    _Properties.MITOSIS_QUALIFICATION:    1,
    _Properties.POSITION:                 3,
    _Properties.INPUTS:                   8,
    _Properties.OUTPUTS:                  8,
}


def _get_indexing(property_name: _Properties) -> Union[int, slice]:
  property_names, property_sizes = list(_DATA_SIZES.keys()), list(_DATA_SIZES.values())
  property_index = property_names.index(property_name)
  property_size = property_sizes[property_index]
  data_format_start_index = sum(property_sizes[:property_index], 0)
  return data_format_start_index if property_size == 1 else slice(data_format_start_index, data_format_start_index + property_size)
  

class Data(Enum):
    ACTIVATION_THRESHOLD     = _get_indexing(_Properties.ACTIVATION_THRESHOLD)
    ACTIVATION_COOLDOWN      = _get_indexing(_Properties.ACTIVATION_COOLDOWN)
    SIGNAL_STRENGTH          = _get_indexing(_Properties.SIGNAL_STRENGTH)
    LATENT_STATE             = _get_indexing(_Properties.LATENT_STATE)
    HORMONE_INFLUENCE        = _get_indexing(_Properties.HORMONE_INFLUENCE)
    TOTAL_RECEPTIVITY        = _get_indexing(_Properties.TOTAL_RECEPTIVITY)
    TOTAL_EMISSIVITY         = _get_indexing(_Properties.TOTAL_EMISSIVITY)
    ACTIVATION_PROGRESS      = _get_indexing(_Properties.ACTIVATION_PROGRESS)
    MITOSIS_QUALIFICATION    = _get_indexing(_Properties.MITOSIS_QUALIFICATION)
    POSITION                 = _get_indexing(_Properties.POSITION)
    INPUTS                   = _get_indexing(_Properties.INPUTS)
    OUTPUTS                  = _get_indexing(_Properties.OUTPUTS)


class Specimen:
  pass
